In [2]:
#Make necessary imports
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import os, sys
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import seaborn as sns
sns.set()

In [3]:
df = pd.read_csv(r"C:\Users\tejas\Downloads\kidney_disease.csv")[['age','wc','htn','dm','classification']]
df.head(n=10)

,age,wc,htn,dm,classification
0,48.0,7800,yes,yes,ckd
1,7.0,6000,no,no,ckd
2,62.0,7500,no,yes,ckd
3,48.0,6700,yes,no,ckd
4,51.0,7300,no,no,ckd
5,60.0,7800,yes,yes,ckd
6,68.0,NaN,no,no,ckd
7,24.0,6900,no,yes,ckd
8,52.0,9600,yes,yes,ckd
9,53.0,12100,yes,yes,ckd


In [4]:
#find null Values in the data set

df.isnull().sum()

age                 9
wc                105
htn                 2
dm                  2
classification      0
dtype: int64

In [5]:
# Impuing Null values

from sklearn.impute import SimpleImputer
imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

df_imputed=pd.DataFrame(imp_mode.fit_transform(df))
df_imputed.columns=df.columns
df_imputed

,age,wc,htn,dm,classification
0,48.0,7800,yes,yes,ckd
1,7.0,6000,no,no,ckd
2,62.0,7500,no,yes,ckd
3,48.0,6700,yes,no,ckd
4,51.0,7300,no,no,ckd
...,...,...,...,...,...
395,55.0,6700,no,no,notckd
396,42.0,7800,no,no,notckd
397,12.0,6600,no,no,notckd
398,17.0,7200,no,no,notckd


In [6]:
# Finding Unique values in the columns

for i in df_imputed.columns:
    print("*************************************************",i,"************************************************************")
    print()
    print(set(df_imputed[i].tolist()))
    print()

************************************************* age ************************************************************

{2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 11.0, 12.0, 14.0, 15.0, 17.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 90.0}

************************************************* wc ************************************************************

{'6500', '19100', '7900', '11200', '8100', '11400', '10700', '8600', '8000', '7100', '7500', '10900', '5100', '12400', '9100', '11000', '12500', '9700', '7800', '5600', '\t?', '8300', '8500', '4700', '7000', '10800', '12300', '4900', '12200', '6200', '12100', '6600', '26400', '18900', '4200', '8200', '12800', '8800',

In [7]:
df_imputed["classification"]=df_imputed["classification"].apply(lambda x:'ckd' if x=="ckd\t"  else x)

df_imputed["dm"]=df_imputed["dm"].apply(lambda x:'no' if x=="\tno"  else x)
df_imputed["dm"]=df_imputed["dm"].apply(lambda x:'yes' if x=="\tyes"  else x)
df_imputed["dm"]=df_imputed["dm"].apply(lambda x:'yes' if x==' yes'  else x)


df_imputed["wc"]=df_imputed["wc"].apply(lambda x:'9800' if x=='\t6200'  else x)
df_imputed["wc"]=df_imputed["wc"].apply(lambda x:'9800' if x=='\t8400'  else x)
df_imputed["wc"]=df_imputed["wc"].apply(lambda x:'9800' if x== '\t?'  else x)



In [8]:
# Finding Unique values in the columns

for i in df_imputed.columns:
    print("*************************************************",i,"************************************************************")
    print()
    print(set(df_imputed[i].tolist()))
    print()

************************************************* age ************************************************************

{2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 11.0, 12.0, 14.0, 15.0, 17.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 90.0}

************************************************* wc ************************************************************

{'6500', '19100', '7900', '11200', '8100', '11400', '10700', '8600', '8000', '7100', '7500', '10900', '5100', '12400', '9100', '11000', '12500', '9700', '7800', '5600', '8300', '8500', '4700', '7000', '10800', '12300', '4900', '12200', '6200', '12100', '6600', '26400', '18900', '4200', '8200', '12800', '8800', '10300

In [9]:
df_imputed["wc"]=df_imputed["wc"].apply(lambda x: float(x))

In [10]:
df_imputed.dtypes

age                object
wc                float64
htn                object
dm                 object
classification     object
dtype: object

In [11]:
from sklearn import preprocessing

df_enco=df_imputed.apply(preprocessing.LabelEncoder().fit_transform)
df_enco

,age,wc,htn,dm,classification
0,40,35,1,1,0
1,5,19,0,0,0
2,54,33,0,1,0
3,40,25,1,0,0
4,43,31,0,0,0
...,...,...,...,...,...
395,47,25,0,0,1
396,34,35,0,0,1
397,8,24,0,0,1
398,11,30,0,0,1


In [12]:
# Lets make some final changes to the data 

# Seperate independent and dependent variables and drop the ID column
x=df_enco.drop(["classification"],axis=1)
y=df_enco["classification"]

In [13]:
# Lets detect the label balance
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

print(Counter(y))

Counter({0: 250, 1: 150})


In [14]:
# Lets balance the labels

ros = RandomOverSampler()

X_ros, y_ros = ros.fit_resample(x, y)
print(Counter(y_ros))

Counter({0: 250, 1: 250})


In [15]:
#Initialize a MinMaxScaler and scale the features to between -1 and 1 to normalize them. 
#The MinMaxScaler transforms features by scaling them to a given range.
#The fit_transform() method fits to the data and then transforms it. We don’t need to scale the labels.
#Scale the features to between -1 and 1

# Scaling is important in the algorithms such as support vector machines (SVM) and k-nearest neighbors (KNN) where distance 
# between the data points is important.

scaler=MinMaxScaler((-1,1))
x=scaler.fit_transform(X_ros)
y=y_ros

In [16]:
#Now,split the dataset into training and testing sets keeping 20% of the data for testing.
#Split the dataset
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x, y, test_size=0.2, random_state=7)

In [17]:
from sklearn.ensemble import RandomForestClassifier 
model = RandomForestClassifier(max_depth=4, random_state = 10) 
model.fit(x_train, y_train)

RandomForestClassifier(max_depth=4, random_state=10)

In [18]:
from sklearn.metrics import accuracy_score
pred_cv = model.predict(x_test)
accuracy_score(y_test,pred_cv)

0.89

In [19]:
# saving the model 
import pickle 
pickle_out = open("classifier.pkl", mode = "wb") 
pickle.dump(model, pickle_out) 
pickle_out.close()

In [20]:
df_imputed.columns

Index(['age', 'wc', 'htn', 'dm', 'classification'], dtype='object')

In [21]:
%%writefile app.py
 
import pickle
import streamlit as st
 
# loading the trained model
pickle_in = open('classifier.pkl', 'rb') 
classifier = pickle.load(pickle_in)
 
@st.cache()
  
# defining the function which will make the prediction using the data which the user inputs 
def prediction(age, wc, htn, dm):   
 
    # Pre-processing user input    
    if htn == "no":
        htn = 0
    else:
        htn = 1
 
    if dm == "no":
        dm = 0
    else:
        dm = 1
 
      
    # Making predictions 
    prediction = classifier.predict( 
        [[age, wc, htn, dm]])
     
    if prediction == 0:
        pred = 'Kidney Disease not detected'
    else:
        pred = 'Kidny Disease found'
    return pred
      
  
# this is the main function in which we define our webpage  
def main():       
    # front end elements of the web page 
    html_temp = """ 
    <div style ="background-color:cyan;padding:13px"> 
    <h1 style ="color:black;text-align:center;">Kidney Disease Prediction</h1> 
    </div> 
    """
      
    # display the front end aspect
    st.markdown(html_temp, unsafe_allow_html = True) 
      
    # following lines create boxes in which user can enter data required to make prediction 
    htn = st.selectbox('htn',("no","yes"))
    dm = st.selectbox('dm Status',("no","yes")) 
    age = st.number_input("AGE") 
    wc = st.number_input("WC")
    result =""
      
    # when 'Predict' is clicked, make the prediction and store it 
    if st.button("Predict"): 
        result = prediction(age, wc, htn, dm) 
        st.success('Report Results: {}'.format(result))
        
     
if __name__=='__main__': 
    main()

Overwriting app.py
